In [1]:
import scrapy
import time
from selenium import webdriver
DRIVER_PATH ='/Users/Elitebook/Downloads/chromedriver_win32/chromedriver.exe'
def fetch_image_urls(query:'iconography', max_links_to_fetch:100, wd:DRIVER_PATH, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q='iconography'))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [2]:
def persist_image(folder_path:'/Users/Elitebook/Desktop/image',url:str):
    try:
        image_content = requests.get(url,verify=False).content
    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")


In [3]:
import os
import io
from PIL import Image
import hashlib
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
def search_and_download(search_term:'iconography',driver_path:str,target_path='C:/Users/Elitebook/Desktop/image',number_images=50):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [4]:
search_term = 'iconography'
search_and_download(search_term = search_term,
                    driver_path = DRIVER_PATH)
                    
                    

Found: 90 search results. Extracting links from 0:90
Found: 50 image links, done!
SUCCESS - saved https://i.pinimg.com/originals/e2/68/09/e26809f3e8881b8c29b78987e7dd8369.png - as C:/Users/Elitebook/Desktop/image\iconography\37bfd24ae7.jpg
SUCCESS - saved https://cdn.dribbble.com/users/44323/screenshots/14956683/media/a24d9d8b55c4e7d9c68258cf39c80854.png?compress=1&resize=400x300 - as C:/Users/Elitebook/Desktop/image\iconography\540739910b.jpg
ERROR - Could not save https://www.mozaik.com/blog/wp-content/uploads/aia_pictograms_01-576x700.jpg - cannot identify image file <_io.BytesIO object at 0x0000021440B54270>
SUCCESS - saved https://cdn.dribbble.com/users/62525/screenshots/14550994/icons_dribbble_no_handles_hb.png?compress=1&resize=400x300 - as C:/Users/Elitebook/Desktop/image\iconography\87e519ff53.jpg
SUCCESS - saved https://graphicburger.com/wp-content/uploads/2016/11/spaceiconography-300.jpg - as C:/Users/Elitebook/Desktop/image\iconography\13a98732ab.jpg
SUCCESS - saved https:/

SUCCESS - saved https://brand.opentable.com/wp-content/uploads/2019/11/Frame-2.png - as C:/Users/Elitebook/Desktop/image\iconography\a0d51ecff2.jpg
SUCCESS - saved https://image.shutterstock.com/image-vector/vector-flat-transparent-icons-design-260nw-460485022.jpg - as C:/Users/Elitebook/Desktop/image\iconography\5803ea749f.jpg
SUCCESS - saved https://go.forrester.com/wp-content/uploads/2020/11/icons.png - as C:/Users/Elitebook/Desktop/image\iconography\2f1ac32ea5.jpg
SUCCESS - saved https://2672686a4cf38e8c2458-2712e00ea34e3076747650c92426bbb5.ssl.cf1.rackcdn.com/2019-07-17-10-02-17.jpeg - as C:/Users/Elitebook/Desktop/image\iconography\e3136eba91.jpg
SUCCESS - saved https://miro.medium.com/max/2000/1*A2Zc-Muv9rxHCbIPwAt9UA@2x.png - as C:/Users/Elitebook/Desktop/image\iconography\95a54d9941.jpg
